### Emily Smith - 44008071

### DATA422-23S2:  Data Wrangling

#### **Group Project**:  Scraping UN Data

*last updated: 27 October 2023*

Adapted from my Assignment 2 project concerning Formula 1 driver statistics.  This was one of my key responsibilities on the group project and does not contain work from other team members.

# Add packages

In [1]:
import Pkg
using Pkg
Pkg.add("Queryverse")
Pkg.add("ExcelFiles")
Pkg.add("HTTP")
Pkg.add("Gumbo")
Pkg.add("DataFrames")
Pkg.add("Cascadia")
Pkg.add("CSV")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Get one table from one country using full link & selector

In [78]:
using HTTP
using Gumbo
using Cascadia
using DataFrames

# This function takes the strings link and css_selector (not a Selector object!)
# the page is parsed, the css_selector grabs only one table, and then each row is itterated through
# Only the first column (Variable) and last column (Value) are retained.
# These two vectors become columns in one DataFrame 'df' which is returned.
# This returns one dataframe representing one table from one css_selector
function get_country_data(link, css_selector)
    # Send an HTTP GET request and parse the response
    response = HTTP.get(link)
    body = String(response.body)
    parsed_html = parsehtml(body)
    
    # Use the Cascadia selector to extract the desired table
    selected_table = eachmatch(Selector(css_selector), parsed_html.root)[1]
    
    # Initialize empty vectors to store "Variable" and "Value" columns
    variables = String[]
    values = String[]
    
    # Iterate through table rows and extract data from the first and last columns
    for row in eachmatch(Selector("tr"), selected_table)
        columns = eachmatch(Selector("td"), row)
        variable = strip(nodeText(first(columns)))  # Remove leading/trailing spaces
        value = strip(nodeText(last(columns)))      # Remove leading/trailing spaces
        
        # Skip the row if the variable is empty or contains only spaces
        if !isempty(variable)
            push!(variables, variable)
            push!(values, value)
        end
    end
    
    # Create a dataframe with the extracted "Variable" and "Value" columns
    df = DataFrame(Variable = variables, Value = values)
    
    return df
end

get_country_data (generic function with 1 method)

### Example output from one table from one country using a full link & selector

In [87]:
example_link = "https://data.un.org/en/iso/br.html"
example_css_selector = ".pure-menu-list > details:nth-child(1) > table:nth-child(2)"

get_country_data(example_link, example_css_selector)

Row,Variable,Value
,String,String
1,Region,South America
2,"Population (000, 2021)",213 993a
3,"Pop. density (per km2, 2021)",25.6a
4,Capital city,Brasilia
5,"Capital city pop. (000, 2021)","4 559.0c,b"
6,UN membership date,24-Oct-45
7,Surface area (km2),8 515 767b
8,Sex ratio (m per 100 f),96.5a
9,National currency,Brazilian Real (BRL)


# Get all three-or-four tables from one country & one local link

In [79]:
using Gumbo
using HTTP
using Cascadia
using DataFrames

# This function takes the strings country_name and local_link
# the local link is the local address on the website, which will be combined
# to make the full usuable url to create a full link for get_country_data
# The function calls get_country_data for each of the four tables on most 
# pages, however if any tables are missing, the try/catch statement handles 
# the error gracefully, as there are two known exceptions.
# the three-or-four tables are concatinated and returned as combined_df
function get_all_tables(country_name, local_link)
    # Combine the URL
    url = "https://data.un.org/$local_link"
    
    selectors = [
        ".pure-menu-list > details:nth-child(1) > table:nth-child(2)",
        ".pure-menu-list > details:nth-child(2) > table:nth-child(2)",
        ".pure-menu-list > details:nth-child(3) > table:nth-child(2)",
        ".pure-menu-list > details:nth-child(4) > table:nth-child(2)"
    ]
    
    result_dfs = DataFrame[]
    
    for selector in selectors
        try
            example_df = get_country_data(url, selector)
            push!(result_dfs, insertcols!(example_df, 1, :Country => country_name))
        catch
            # Handle the expected error (selector not found) and continue to the next selector
        end
    end
    
    combined_df = vcat(result_dfs...)
    
    return combined_df
end


get_all_tables (generic function with 2 methods)

### Example with four tables

In [80]:
example_country_name = "Algeria"
example_link = "en/iso/dz.html"

result_df = get_all_tables(example_country_name, example_link)
println(result_df)

56×3 DataFrame
 Row │ Country  Variable                           Value                
     │ String   String                             String               
─────┼──────────────────────────────────────────────────────────────────
   1 │ Algeria  Region                             Northern Africa
   2 │ Algeria  Population (000, 2021)             44 617a
   3 │ Algeria  Pop. density (per km2, 2021)       18.7a
   4 │ Algeria  Capital city                       Algiers
   5 │ Algeria  Capital city pop. (000, 2021)      2 729.3c,b
   6 │ Algeria  UN membership date                 08-Oct-62
   7 │ Algeria  Surface area (km2)                 2 381 741b
   8 │ Algeria  Sex ratio (m per 100 f)            102.1a
   9 │ Algeria  National currency                  Algerian Dinar (DZD)
  10 │ Algeria  Exchange rate (per US$)            132.1d
  11 │ Algeria  GDP: Gross domestic product (mil…  171 158b
  12 │ Algeria  GDP growth rate (annual %, const…  0.8b
  13 │ Algeria  GDP per capita (cur

### Example with three tables

In [81]:
example_country_name = "Vatican"
example_link = "en/iso/va.html"

result_df = get_all_tables(example_country_name, example_link)
println(result_df)

15×3 DataFrame
 Row │ Country  Variable                           Value             
     │ String   String                             String            
─────┼───────────────────────────────────────────────────────────────
   1 │ Vatican  Region                             Southern Europe
   2 │ Vatican  Population (000, 2021)             1a,b
   3 │ Vatican  Pop. density (per km2, 2021)       1 845.5a,b
   4 │ Vatican  Capital city                       Vatican City
   5 │ Vatican  Capital city pop. (000, 2021)      0.8e
   6 │ Vatican  Surface area (km2)                 ~0c,d
   7 │ Vatican  National currency                  Euro (EUR)
   8 │ Vatican  Exchange rate (per US$)            0.8f
   9 │ Vatican  Population growth ratea,g (avera…  0.1f
  10 │ Vatican  Urban population (% of total pop…  100d
  11 │ Vatican  Urban population growth rateg (a…  ...
  12 │ Vatican  International migrant stocka,h (…  0.8 /      100.0f
  13 │ Vatican  Intentional homicide rate (per 1…  ...
  14

# Get the dataframe of all country names & local links

In [82]:
using Gumbo
using HTTP
using Cascadia
using DataFrames

# This function takes the Strings link and link_prefix
# to find all link elements on a scraped page, and then step through each 
# item to only investigate links contianing the link_prefix
# The links returned are strings containing only the local 'stub' of the url,
# this is combined with the associated country name and combined in a dataframe
# The dataframe 'df' of country names and local links is returned
function extract_country_and_links(link, link_prefix)
    # Send an HTTP GET request and parse the response
    response = HTTP.get(link)
    html = String(response.body)
    parsed_html = parsehtml(html)
    
    # Find all <a> elements on the page
    a_elements = eachmatch(Selector("a"), parsed_html.root)
    
    # Initialize empty arrays to store country names and links
    country_names = String[]
    country_links = String[]
    
    # Iterate through <a> elements and filter out the ones with desired links
    for a_element in a_elements
        # Extract the link from the <a> element
        country_link = getattr(a_element, "href")
        
        # Check if the link starts with the specified prefix
        if startswith(country_link, link_prefix)
            # Extract the country name from the <a> element text
            country_name = nodeText(a_element)
            
            # Append the country name and link to the respective arrays
            push!(country_names, country_name)
            push!(country_links, country_link)
        end
    end

    # Create a dataframe from the extracted data
    df = DataFrame(Country = country_names, Link = country_links)
    
    return df
end

# Link and link prefix
country_list_link = "https://data.un.org/"
link_prefix = "en/iso/"

# Call the function to extract country names and links
country_list_df = extract_country_and_links(country_list_link, link_prefix)

# Print the resulting dataframe
println(country_list_df)

232×2 DataFrame
 Row │ Country                            Link           
     │ String                             String         
─────┼───────────────────────────────────────────────────
   1 │ Afghanistan                        en/iso/af.html
   2 │ Albania                            en/iso/al.html
   3 │ Algeria                            en/iso/dz.html
   4 │ American Samoa                     en/iso/as.html
   5 │ Andorra                            en/iso/ad.html
   6 │ Angola                             en/iso/ao.html
   7 │ Anguilla                           en/iso/ai.html
   8 │ Antigua and Barbuda                en/iso/ag.html
   9 │ Argentina                          en/iso/ar.html
  10 │ Armenia                            en/iso/am.html
  11 │ Aruba                              en/iso/aw.html
  12 │ Australia                          en/iso/au.html
  13 │ Austria                            en/iso/at.html
  14 │ Azerbaijan                         en/iso/az.html
  15 │ Baham

# WARNING - NEXT CELL SCRAPES ALL COUNTRIES

## Automate through all countries & local links

In [83]:
# This line itterates through all countries/links in country_list_df
# and creates all_dfs as a dataframe filled with each country's seperate dataframe
# NEARLY 300 COUNTRIES
all_dfs = map(row -> get_all_tables(row.Country, row.Link), eachrow(country_list_df))

232-element Vector{DataFrame}:
 52×3 DataFrame
 Row │ Country      Variable                           Value              
     │ String       String                             String             
─────┼────────────────────────────────────────────────────────────────────
   1 │ Afghanistan  Region                             Southern Asia
   2 │ Afghanistan  Population (000, 2021)             39 835a
   3 │ Afghanistan  Pop. density (per km2, 2021)       61a
   4 │ Afghanistan  Capital city                       Kabul
   5 │ Afghanistan  Capital city pop. (000, 2021)      4 114.0b
   6 │ Afghanistan  UN membership date                 19-Nov-46
   7 │ Afghanistan  Surface area (km2)                 652 864b
   8 │ Afghanistan  Sex ratio (m per 100 f)            105.3a
   9 │ Afghanistan  National currency                  Afghani (AFN)
  10 │ Afghanistan  Exchange rate (per US$)            77.1c
  11 │ Afghanistan  GDP: Gross domestic product (mil…  17 877b
  ⋮  │      ⋮               

### Combine the above dataframe-of-dataframes into one dataframe

In [84]:
# This combines all_dfs into one 11475×3 dataframe
final_country_data = vcat(all_dfs...)

Row,Country,Variable,Value
,String,String,String
1,Afghanistan,Region,Southern Asia
2,Afghanistan,"Population (000, 2021)",39 835a
3,Afghanistan,"Pop. density (per km2, 2021)",61a
4,Afghanistan,Capital city,Kabul
5,Afghanistan,"Capital city pop. (000, 2021)",4 114.0b
6,Afghanistan,UN membership date,19-Nov-46
7,Afghanistan,Surface area (km2),652 864b
8,Afghanistan,Sex ratio (m per 100 f),105.3a
9,Afghanistan,National currency,Afghani (AFN)


### Export as CSV

In [85]:
using CSV

# export the final_country_data for use by other team members
CSV.write("final_country_data.csv", final_country_data, writeheader=true)

"final_country_data.csv"